In [72]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [73]:
import pandas as pd
import os

In [74]:
files=[file for dirpath,directory,file in os.walk('./all_csv/')][0]
# files=[file for dirpath,directory,file in os.walk(r'/gdrive/My Drive/all_csv/')][0]

In [75]:
match_data=pd.DataFrame(data=None)
odi_scorecard=pd.DataFrame(data=None)
ttwenty_scorecard=pd.DataFrame(data=None)
odi_info=pd.DataFrame(data=None)
ttwenty_info=pd.DataFrame(data=None)

In [76]:
def rename_date_umpire(index_list):
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='date':
            index_list[i]+='_'+str(n)
            n+=1
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='umpire':
            index_list[i]+='_'+str(n)
            n+=1
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='team':
            index_list[i]+='_'+str(n)
            n+=1
    return index_list

In [77]:
def find_game(df_game,df_info):
    if 'series' in df_info.columns:
        if 'odi' in str.lower(df_info.iloc[0]['series']):
            return 'odi'
        if 't20i' in str.lower(df_info.iloc[0]['series']) or 't20' in str.lower(df_info.iloc[0]['series']) or 'indian premier league' in str.lower(df_info.iloc[0]['series']) :
            return 'twenty'
    if max(df_game['balls-bowled'])<=24:
        return 'twenty'
    if 24<max(df_game['balls-bowled'])<=60:
        return 'odi'
    return

In [78]:
def append_file(temp_df,temp_info_df,type_game):
    global odi_scorecard
    global ttwenty_scorecard
    global odi_info
    global ttwenty_info
    if type_game=='odi':
        odi_scorecard=odi_scorecard.append(temp_df,ignore_index=True)
        odi_info=odi_info.append(temp_info_df,ignore_index=True)
    elif type_game=='twenty':
        ttwenty_scorecard=ttwenty_scorecard.append(temp_df,ignore_index=True)
        ttwenty_info=ttwenty_info.append(temp_info_df,ignore_index=True)

In [79]:
def get_extras_type(match_data):
    list_extras=[]
    for index,row in match_data.iterrows():
        ov=str(row['over'])
        if '.' in ov:
            ov=str(row['over']).split('.')
            ball_no=int(ov[1])
            over_no=int(ov[0])
        else:
            continue
        if row['extras']!=0:
            if row['runs']!=0:
                match_data.loc[index,'extras_type']='w'
            list_extras.append(index)
        if ball_no>6:
            if len(list_extras)>0:
                match_data.loc[list_extras.pop(-1),'extras_type']='w'
    for i in list_extras:
        match_data.loc[i,'extras_type']='b'
    return match_data

In [80]:
def prepare_scorecard(match_data):
    match_data=get_extras_type(match_data)
#     print(match_data[match_data['bowler']==	'Mashrafe Mortaza'])
    players=list((match_data['striker'].append(match_data['non-striker']).append(match_data['bowler'])).unique())
    #     to make 22 players if any player has not played
#     for i in range(len(players),22):
#         players.append('p_'+str(i))
#         fow
#     player_stats=['match-id','innings','name','batting-position','over-batsman','runs-scored','balls-played','ones','twos','threes','fours','sixes','wicket-method','balls-bowled','maiden-overs','runs-given','wickets',extras,'fall-of-wicket-score','fall-of-wicket-overs','fall-of-wicket-no','fow','fow_runs','fow_overs','fow_batsman','fow_bowler']
    player_stats=['match-id','innings','name','batting-position','over-batsman','runs-scored','balls-played','dots','ones','twos','threes','fours','sixes','wicket-method','balls-bowled','maiden-overs','runs-given','wickets','extras','fall-of-wicket-score','fall-of-wicket-overs','fall-of-wicket-no','fall-of-wicket-bowler']
    player_data={key:{key_type:0 for key_type in player_stats} for key in players}
    for p in players:
        player_data[p]['match-id']=match_data.loc[0,'file_no']
        player_data[p]['name']=p
    team_score=0
    balls=0
    pos=1
    inning=False
    w=1
    p_no=1
    w_no=1
    extras_over=0
    for index, row in match_data.iterrows():
        ov=str(row['over'])
        if '.' in ov:
            ov=ov.split('.')
            ball_no=int(ov[1])
            over_no=int(ov[0])
        else:
            continue
        if over_no>50:
            player_data=[value for key, value in player_data.items()]
            scorecard=pd.DataFrame(data=player_data)
            scorecard=scorecard[player_stats]
            return scorecard
        if ball_no==1 and over_no==0:
            pos=1
            w=1
            team_score=0
            w_no=1
            runs_over=0
            # if row['innings']!=1:
            #     p_no=12
        if ball_no==1:
            extras_over=0
        if row['runs']==1:
            player_data[row['striker']]['ones']+=1
        elif row['runs']==2:
            player_data[row['striker']]['twos']+=1
        elif row['runs']==3:
            player_data[row['striker']]['threes']+=1
        elif row['runs']==4:
            player_data[row['striker']]['fours']+=1
        elif row['runs']==6:
            player_data[row['striker']]['sixes']+=1
        elif row['extras']==0:
            player_data[row['striker']]['dots']+=1

        if player_data[row['striker']]['batting-position']==0:
            player_data[row['striker']]['batting-position']=pos
            player_data[row['striker']]['over-batsman']=row['over']-extras_over
            pos+=1
        if player_data[row['non-striker']]['batting-position']==0:
            player_data[row['non-striker']]['batting-position']=pos
            player_data[row['non-striker']]['over-batsman']=row['over']-extras_over
            pos+=1
    # wicket
#         print(row['out-player'])
        if not pd.isna(row['out-player']):
            player_data[row['out-player']]['wicket-method']=row['out']
#             fow
#             player_data[players[p_no-1]]['fow']=w
#             player_data[players[p_no-1]]['fow_runs']=team_score
#             player_data[players[p_no-1]]['fow_overs']=row['over']
#             player_data[players[p_no-1]]['fow_batsman']=row['out-player']
#             player_data[players[p_no-1]]['fow_bowler']=row['bowler']
            p_no+=1
            w+=1
            if row['out']!='run out':
                player_data[row['bowler']]['wickets']+=1
            player_data[row['out-player']]['fall-of-wicket-score']=team_score
            player_data[row['out-player']]['fall-of-wicket-overs']=row['over']-extras_over
            print(row['over'],extras_over)
            player_data[row['out-player']]['fall-of-wicket-no']=w_no
            player_data[row['out-player']]['fall-of-wicket-bowler']=row['bowler']
            w_no+=1
        team_score+=row['runs']+row['extras']
        runs_over+=row['runs']
        if row['extras']!=0 and row['extras_type']=='w':
            runs_over+=1
            player_data[row['bowler']]['extras']+=1
            player_data[row['striker']]['runs-scored']+=row['extras']-1
            player_data[row['bowler']]['runs-given']+=row['extras']-1
            extras_over+=0.1
        elif row['extras']!=0:
#             print(row)
            player_data[row['bowler']]['balls-bowled']+=1
            player_data[row['striker']]['balls-played']+=1
        else:
            player_data[row['striker']]['balls-played']+=1
            player_data[row['bowler']]['balls-bowled']+=1
            player_data[row['bowler']]['runs-given']+=row['runs']
        player_data[row['striker']]['runs-scored']+=row['runs']
        if ball_no>=6:
            if ball_no==6 and runs_over==0:
                player_data[row['bowler']]['maiden-overs']+=1
            runs_over=0
        player_data[row['striker']]['innings']=row['innings']
        if row['innings']==1:
            player_data[row['bowler']]['innings']=2
        else:
            player_data[row['bowler']]['innings']=1
#     print(player_data)
    player_data=[value for key, value in player_data.items()]
    scorecard=pd.DataFrame(data=player_data)
    scorecard=scorecard[player_stats]
    return scorecard

In [81]:
i=0
for file in files[:1]:
    print(i,'--',file)
    i+=1
    count=0
    df_index=[]
    df_row=[]
    add=r'./all_csv/'+file
#     add=r'/gdrive/My Drive/all_csv/'+file
    df_index=['file_no']
    file_no=file.split('.')[0]
    df_row=[file_no]
    with open(add) as f:
        new_f=f.readlines()
        for line in new_f:
            if 'version' in line:
                count+=1
            elif 'info' in line:
                count+=1
                line=line.strip().split(',')
                df_index.append(line[1])
                df_row.append(line[2])
            else:
                df_index=rename_date_umpire(df_index)
                df_dic=dict(zip(df_index,df_row),index=[0])
                temp_info_df=pd.DataFrame(df_dic)
#                 df_info=df_info.append(temp_info_df,ignore_index=True)
                # gender=df_info['gender'].iloc[0]
                # gender=str.lower(gender.strip())
                break
    temp_df=pd.read_csv(add,skiprows=count,names=[0,'innings','over','batting-team','striker','non-striker','bowler','runs','extras','out','out-player'])
    temp_df=temp_df.drop([0],axis=1)
    temp_df['file_no']=[file_no]*(temp_df.shape[0])
    temp_sc=prepare_scorecard(temp_df)
    # print(temp_sc)
#     append_file(temp_df,gender,type_game) 
    append_file(temp_sc,temp_info_df,find_game(temp_sc,temp_info_df))

0 -- 1019975.csv
0.4 0
10.5 0
13.1 0
22.5 0
25.3 0
37.3 0
42.5 0
45.2 0
46.6 0.1
49.6 0
7.2 0
22.7 0.1
25.3 0
28.5 0.1
30.4 0
31.3 0
32.4 0
36.4 0
39.3 0
42.5 0


In [82]:
odi_info.columns

Index(['file_no', 'team_0', 'team_1', 'gender', 'season', 'date_0', 'series',
       'match_number', 'venue', 'city', 'toss_winner', 'toss_decision',
       'player_of_match', 'umpire_0', 'umpire_1', 'reserve_umpire',
       'tv_umpire', 'match_referee', 'winner', 'winner_runs', 'index'],
      dtype='object')

In [83]:
ttwenty_info.columns

Index([], dtype='object')

In [84]:
ttwenty_info

""


In [85]:
ttwenty_scorecard

""


In [86]:
odi_info

,file_no,team_0,team_1,gender,season,date_0,series,match_number,venue,city,...,toss_decision,player_of_match,umpire_0,umpire_1,reserve_umpire,tv_umpire,match_referee,winner,winner_runs,index
0,1019975,New Zealand,Bangladesh,male,2016/17,2016/12/29,Bangladesh in New Zealand ODI Series,2,Saxton Oval,Nelson,...,field,NT Broom,CM Brown,PR Reiffel,WR Knights,C Shamshuddin,BC Broad,New Zealand,67,0


In [87]:
odi_scorecard['fall-of-wicket-overs'].value_counts()

25.3    2
0.0     2
42.5    2
28.4    1
22.5    1
46.5    1
7.2     1
30.4    1
13.1    1
39.3    1
22.6    1
32.4    1
45.2    1
31.3    1
36.4    1
37.3    1
0.4     1
49.6    1
10.5    1
Name: fall-of-wicket-overs, dtype: int64

In [149]:
odi_info.to_csv('./odi_info.csv',index=False)
ttwenty_info.to_csv('./ttwenty_info.csv',index=False)
odi_scorecard.to_csv('./odi_scorecard.csv',index=False)
ttwenty_scorecard.to_csv('./ttwenty_scorecard.csv',index=False)